In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


c:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:

# Directories
train_dir = "DataSet_V1_8020/train"
val_dir = "DataSet_V1_8020/val"
test_dir = "DataSet_V1_8020/test"


In [3]:

# Data augmentation and rescaling for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Rescaling for validation and test
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Train data
train_data = train_datagen.flow_from_directory(
    directory=train_dir,
    batch_size=32,
    target_size=(299, 299),
    class_mode="categorical",
    shuffle=True  # Ensure shuffling
)

# Validation data
val_data = val_datagen.flow_from_directory(
    directory=val_dir,
    batch_size=32,
    target_size=(299, 299),
    class_mode="categorical"
)

# Test data
test_data = test_datagen.flow_from_directory(
    directory=test_dir,
    batch_size=32,
    target_size=(299, 299),
    class_mode="categorical",
    shuffle=False  # No need to shuffle test data
)


Found 4434 images belonging to 14 classes.
Found 1115 images belonging to 14 classes.
Found 1393 images belonging to 14 classes.


In [5]:
# Steps per epoch
steps_per_epoch = train_data.samples // train_data.batch_size
validation_steps = val_data.samples // val_data.batch_size

# Number of classes
num_classes = train_data.num_classes
print(f"Number of classes: {num_classes}")

# 1. Create a base model with InceptionV3 (pre-trained)
base_model = tf.keras.applications.InceptionV3(include_top=False, input_shape=(299, 299, 3))
base_model.trainable = False  # Freeze base model

# 2. Build custom model
inputs = tf.keras.layers.Input(shape=(299, 299, 3), name="input-layer")
x = base_model(inputs)
x = tf.keras.layers.GlobalAveragePooling2D(name="global_average_pooling_layer")(x)
outputs = tf.keras.layers.Dense(num_classes, activation="softmax", name="output-layer")(x)
model = tf.keras.Model(inputs, outputs)

# 3. Compile model
model.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),  # Smaller learning rate
    metrics=["accuracy"]
)

# 4. Callbacks
# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Model checkpoint to save the best model based on validation loss
checkpoint = ModelCheckpoint("BirdSpecies_best_model.keras", save_best_only=True, monitor='val_loss', mode='min')

# 5. Train the model
history = model.fit(
    train_data,
    epochs=50,  # Increase number of epochs
    steps_per_epoch=steps_per_epoch,
    validation_data=val_data,
    validation_steps=validation_steps,
    callbacks=[early_stopping, checkpoint]
)

Number of classes: 14


C:\Users\Admin\AppData\Roaming\Python\Python39\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 13s/step - accuracy: 0.1984 - loss: 2.5153 

C:\Users\Admin\AppData\Roaming\Python\Python39\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


138/138 ━━━━━━━━━━━━━━━━━━━━ 2115s 15s/step - accuracy: 0.1992 - loss: 2.5135 - val_accuracy: 0.5184 - val_loss: 1.8283
Epoch 2/50
  1/138 ━━━━━━━━━━━━━━━━━━━━ 4:56 2s/step - accuracy: 0.5000 - loss: 1.8910

c:\ProgramData\Anaconda3\lib\contextlib.py:137: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


138/138 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - accuracy: 0.5000 - loss: 1.8910 - val_accuracy: 0.4444 - val_loss: 2.0198
Epoch 3/50
138/138 ━━━━━━━━━━━━━━━━━━━━ 1224s 9s/step - accuracy: 0.5861 - loss: 1.7250 - val_accuracy: 0.7206 - val_loss: 1.3595
Epoch 4/50
138/138 ━━━━━━━━━━━━━━━━━━━━ 8s 42ms/step - accuracy: 0.6250 - loss: 1.5111 - val_accuracy: 0.8148 - val_loss: 1.2833
Epoch 5/50
138/138 ━━━━━━━━━━━━━━━━━━━━ 1212s 9s/step - accuracy: 0.7230 - loss: 1.2904 - val_accuracy: 0.7858 - val_loss: 1.0682
Epoch 6/50
138/138 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - accuracy: 0.7812 - loss: 1.0742 - val_accuracy: 0.7778 - val_loss: 1.2638
Epoch 7/50
138/138 ━━━━━━━━━━━━━━━━━━━━ 1223s 9s/step - accuracy: 0.7699 - loss: 1.0614 - val_accuracy: 0.8153 - val_loss: 0.8995
Epoch 8/50
138/138 ━━━━━━━━━━━━━━━━━━━━ 8s 44ms/step - accuracy: 0.6875 - loss: 1.0582 - val_accuracy: 0.8889 - val_loss: 0.8307
Epoch 9/50
138/138 ━━━━━━━━━━━━━━━━━━━━ 1245s 9s/step - accuracy: 0.8014 - loss: 0.9278 - val_accuracy: 0

In [6]:
test_loss, test_accuracy = model.evaluate(test_data)
print(f"Test accuracy: {test_accuracy:.2f}")


44/44 ━━━━━━━━━━━━━━━━━━━━ 318s 7s/step - accuracy: 0.8463 - loss: 0.5970
Test accuracy: 0.87


In [7]:

# 8. Save the final model (best version after fine-tuning)
model.save("TEST_Model_V1_P1.h5")


In [8]:

# 6. Fine-tuning the InceptionV3 model after the initial training
# Unfreeze the base model (only unfreeze the last few layers)
base_model.trainable = True
fine_tune_at = 100  # Fine-tune layers after the 100th layer
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False  # Freeze the first 'fine_tune_at' layers

# Re-compile model with a smaller learning rate
model.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),  # Smaller learning rate for fine-tuning
    metrics=["accuracy"]
)

# Continue training the model with fine-tuning
history_fine = model.fit(
    train_data,
    epochs=20,  # Additional epochs for fine-tuning
    steps_per_epoch=steps_per_epoch,
    validation_data=val_data,
    validation_steps=validation_steps,
    callbacks=[early_stopping, checkpoint]
)


Epoch 1/20
138/138 ━━━━━━━━━━━━━━━━━━━━ 1579s 11s/step - accuracy: 0.8266 - loss: 0.8699 - val_accuracy: 0.9265 - val_loss: 0.2968
Epoch 2/20
138/138 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - accuracy: 0.9062 - loss: 0.3560 - val_accuracy: 0.9259 - val_loss: 0.3389
Epoch 3/20
138/138 ━━━━━━━━━━━━━━━━━━━━ 1571s 11s/step - accuracy: 0.9262 - loss: 0.3473 - val_accuracy: 0.9605 - val_loss: 0.1663
Epoch 4/20
138/138 ━━━━━━━━━━━━━━━━━━━━ 20s 78ms/step - accuracy: 0.9688 - loss: 0.2844 - val_accuracy: 0.9630 - val_loss: 0.1484
Epoch 5/20
138/138 ━━━━━━━━━━━━━━━━━━━━ 1781s 13s/step - accuracy: 0.9527 - loss: 0.2157 - val_accuracy: 0.9752 - val_loss: 0.1061
Epoch 6/20
138/138 ━━━━━━━━━━━━━━━━━━━━ 11s 53ms/step - accuracy: 1.0000 - loss: 0.0704 - val_accuracy: 1.0000 - val_loss: 0.0895
Epoch 7/20
138/138 ━━━━━━━━━━━━━━━━━━━━ 1286s 9s/step - accuracy: 0.9705 - loss: 0.1472 - val_accuracy: 0.9825 - val_loss: 0.0761
Epoch 8/20
138/138 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - accuracy: 0.9688 - loss: 0.1953 -

In [9]:

# 7. Evaluate on test data
test_loss, test_accuracy = model.evaluate(test_data)
print(f"Test accuracy: {test_accuracy:.2f}")


44/44 ━━━━━━━━━━━━━━━━━━━━ 284s 6s/step - accuracy: 0.9913 - loss: 0.0292
Test accuracy: 0.99


In [10]:

# 8. Save the final model (best version after fine-tuning)
model.save("TEST_Model_V1_P2.h5")
